In [2]:
# import pandas as pd
# import pyodbc
# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=kcplsql;'
#                       'Database=KCPL_TBS_Archive;'
#                       'Trusted_Connection=yes;')

# computer = pd.read_sql_query('SELECT * FROM MyPC3SessionAudit WHERE StartTime > ?', conn, parse_dates=['StartTime'], params=['2022-01-01 20:26:15'])
# computer.to_csv('data/computer.csv')
# computer.head(2)

In [4]:
import pandas as pd

computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
circulation = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])

# computer data
mask = computer['SiteName'].isin(['__Not In Use', '_default', '_IS Testing Lab'])
computer = computer[~mask]
replace_values = {'Lucile H. Bluford Branch': 'KC-BLUFORD', 'Plaza Branch': 'KC-PLAZA', 'Central Library': 'KC-CENTRAL', 'Waldo Branch': 'KC-WALDO', 'Southeast Branch': 'KC-SE', 'North-East Branch': 'KC-NE', 'Trails West Branch': 'KC-TRAILS', 'Westport Branch': 'KC-WSTPORT', 'Irene H. Ruiz Biblioteca de las Americas': 'KC-RUIZ', 'Sugar Creek Branch': 'KC-SGCREEK'}
computer = computer.replace({"SiteName": replace_values})
computer1 = computer[['StartTime', 'EndTime', 'SessionID', 'SiteName']]
computer1['Date'] = computer1['StartTime'].dt.date
computer1['time'] = computer1['StartTime'].dt.time
computer1['year'] = computer1['StartTime'].dt.year
computer1['week_day'] = computer1['StartTime'].dt.day_name()
computer1['time'] = computer1['time'].astype(str)
computer1['hour'] = computer1['time'].str.split(':').str[0]
computer1['hour'] = computer1['hour'].astype(int)
computer1['date'] = pd.to_datetime(computer1['Date'])
grouped = computer1.groupby(['date','SiteName', 'hour'])['SessionID'].count().reset_index()
replace_values = {8: '8 AM', 9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM", 21: '9'}
computer_df = grouped.replace({"hour": replace_values})
computer_df.rename(columns={'date': 'Trans Hist Date', 'SiteName': 'Station Library Checkout', 'hour': 'hours'}, inplace=True)
mask1 = circulation['User Profile'].isin(['MISSING', 'KC-DISPLAY', 'KC-MAINT', 'DISCARD', 'KC-STAFF', 'DAMAGED', 'KC-CATALOG', 'KC-SUSPND',
 'LOST', 'KCP-ILL', 'KC-COLDEV', 'KC-TFRBTG', 'KC_CAT1', 'REPLACE'])
circulation = circulation[~mask1]
circulation['trans_time'] = circulation['Trans Hist Datetime'].str.split(' ').str[1]
circulation['hour'] = circulation['trans_time'].str.split(':').str[0]
circulation1 = circulation.groupby(['Trans Hist Date', 'Station Library Checkout', 'hour'])['User Id'].unique().reset_index()
## circulation1 = pd.DataFrame(circulation1)
#circulation1['patron'] = circulation['User Id'].unique()
circulation1['patrons'] = circulation1['User Id'].apply(lambda x: len(x))#.reset_index()
circulation2 = circulation1[['Trans Hist Date', 'Station Library Checkout', 'hour', 'patrons']]
#circulation2['patrons'] = circulation2['patrons'].fillna(0)
circulation2['hours'] = circulation2['hour'].astype(int)
circulation2['week_day'] = circulation2['Trans Hist Date'].dt.day_name()
replace_values = {9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM"}
circulation2 = circulation2.replace({"hours": replace_values})
mer_df = pd.merge(circulation2, computer_df)
mer_df['total'] = mer_df['SessionID'] + mer_df['patrons']

<ipython-input-4-3ef6d4516f8a>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
<ipython-input-4-3ef6d4516f8a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computer1['Date'] = computer1['StartTime'].dt.date
<ipython-input-4-3ef6d4516f8a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computer1['time'] = computer1['StartTime'].dt.time
<ipython-input-4-3ef6d4516f8a>:14: Setti

In [5]:
mer_df['Trans Hist Date'].max()

Timestamp('2022-04-16 00:00:00')